In [1]:
import os
import numpy as np
import tensorflow as tf

2024-09-03 14:26:00.465358: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 14:26:00.469120: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 14:26:00.480138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 14:26:00.498184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 14:26:00.502325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 14:26:00.514995: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
IMG_SIZE = 160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    return image, label

train_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/train", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.1, subset="training", shuffle=True,
    seed=42, batch_size=32
)

valid_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/train", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.1, subset="validation", shuffle=True,
    seed=42, batch_size=32
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    "club-logo/data/test", label_mode = "categorical", image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True, seed=42, batch_size=32
)

train_dataset = train_dataset.map(format_example)
valid_dataset = valid_dataset.map(format_example)
test_dataset = test_dataset.map(format_example)



In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)


In [ ]:
base_model.trainable = False

In [ ]:
next(iter(train_dataset.take(1)))

In [ ]:
image_batch,label_batch = next(iter(train_dataset.take(1)))
print(label_batch)
feature_batch = base_model(image_batch)
print(feature_batch.shape)
feature_batch

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)
feature_batch_average

In [ ]:
prediction_layer = tf.keras.layers.Dense(3)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


In [ ]:
x = base_model.output;
x = global_average_layer(x)
x = prediction_layer(x)


model = tf.keras.Model(inputs=base_model.input,outputs=x)

In [ ]:
base_learning_rate = 0.0001

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
	loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
	metrics=['accuracy']
)

In [ ]:
initial_epochs = 10 # please try `initial_epochs = 10` or `initial_epochs = 20` after `initial_epochs = 2` is okay
validation_steps=20

loss0,accuracy0 = model.evaluate(valid_dataset, steps = validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
import keras

loss_result = []
accuracy_result = []

class PerBatchMetrics(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        loss_result.append(logs['loss'])
        accuracy_result.append(logs['accuracy'])



## The actual training step
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=valid_dataset,
    callbacks=[PerBatchMetrics()]
)

In [ ]:
model.save('my_model.keras', overwrite=True)